In [2]:
from pandas import DataFrame, concat, to_numeric, isnull, NaT, merge, Series, wide_to_long, read_html

import pandas.io.html as ih
import re
import sqlite3

In [3]:
#Not required in functions
import pandas as pd

In [4]:
### PATCHES
#via http://stackoverflow.com/a/28173933/454773
def stringify_children(self,node):
    from lxml.etree import tostring
    from itertools import chain
    parts = ([node.text] +
            list(chain(*([tostring(c, with_tail=False), c.tail] for c in node.getchildren()))) +
            [node.tail])
    txt='\n'.join(str(v.decode()) if isinstance(v, bytes) else str(v) for v in filter(None, parts))
    txt=txt.replace('<br/>','\n')
    txt=re.sub('\n\n+','\n',txt)
    return txt

def nostrip(self, rows):
        """Parse the raw data into a list of lists.
        Parameters
        ----------
        rows : iterable of node-like
            A list of row elements.
        text_getter : callable
            A callable that gets the text from an individual node. This must be
            defined by subclasses.
        column_finder : callable
            A callable that takes a row node as input and returns a list of the
            column node in that row. This must be defined by subclasses.
        Returns
        -------
        data : list of list of strings
        """
        #TH: omit the whitespace stripper from the original function
        data = [[self._text_getter(col) for col in
                 self._parse_td(row)] for row in rows]
        return data

ih._LxmlFrameParser._text_getter=stringify_children
ih._HtmlFrameParser._parse_raw_data=nostrip

def loader(url,**kwargs):
    df=read_html(url,encoding='utf-8',**kwargs)
    #except:
    #    df=[DataFrame()]
    #The original pandas HTML table reader strips whitespace
    #If we disable that as part of gaining access to cell HTML contents, we need to make up for it
    for ddf in df:
        ddf[ddf.select_dtypes(include=['O']).columns]=ddf.select_dtypes(include=['O']).apply(lambda x: x.str.strip())
    return df

def validateLoadedDF(df):
    if (len(df.columns)==1) | (not df.size):
        return False
    return True

In [17]:
def droptable(conn,table):
    cursor = conn.cursor()
    cursor.execute('''DROP TABLE IF EXISTS "{}"'''.format(table))
    conn.commit()

In [6]:
from pandas import isnull, cut, to_timedelta

def partofday(df):
    return cut(df.dt.hour,
                  [-1,5, 12, 17,19, 24],
                  labels=['Overnight','Morning', 'Afternoon', 'Evening','Night'])

def regularTimeString(strtime):
    if isnull(strtime) or strtime=='' or strtime=='nan':
        return to_timedelta(strtime)

    #Go defensive, just in case we're passed eg 0 as an int
    strtime=str(strtime)
    strtime=strtime.strip('+')

    modifier=''
    if strtime.startswith('-'):
        modifier='-'
        strtime=strtime.strip('-')
    timeComponents=strtime.split(':')
    ss=timeComponents[-1]
    mm=timeComponents[-2] if len(timeComponents)>1 else 0
    hh=timeComponents[-3] if len(timeComponents)>2 else 0
    timestr='{}{}:{}:{}'.format(modifier,hh,mm,ss)
    return to_timedelta(timestr)

In [7]:
#orderedRallyCategories=["M","WRC TROPHY","WRC2","WRC3","RGT",'']

driverRE_HTML=r'^\s*<img .* alt="([A-Z]*).*/>\s*(.*)$'
tagcontentRE_HTML= r'^<.*>(.*)</[a-zA-Z]+>$'
tagcontent2RE_HTML=r'^(<.*>)?(.+?)(</[a-zA-Z]+>)?$'
controlRE = r'^[A-Z]*([0-9]*)[A-Z]*$'
stagenumRE=r'^TC([0-9]+)[A-Z]*$'
standing_pointsRE=r'^([0-9\+]*).*\((.*)\).*$'
imgAltRE_HTML= r'^<img.* alt="([^"]*)".*$'


def unicodify(string):
    soup = BeautifulSoup(string,"html5lib")
    return soup.text


def _getTeamCar(df,key='team_Car'):
    return df[key].str.split('\n', 1).str

def _driverNameNationality(df,key='driverName'):
    tmp=df[key].str.extract(driverRE_HTML, expand=True)
    df[key]=tmp[1].str.strip()
    df[key+'_nationality']=tmp[0].str.strip()
    return df[key],df[key+'_nationality']

def _driverCodriverNameNationality(df,key='driver_Codriver'):
    dff=df[key].str.split('\n',expand=True)
    df['driverName']=dff[0]+'\n'+dff[1]
    df['coDriverName']=dff[2]+'\n'+dff[3]
    driver,driverNat=_driverNameNationality(df,key='driverName')
    coDriver,coDriverNat=_driverNameNationality(df,key='coDriverName')
    return driver,driverNat,coDriver,coDriverNat

In [8]:
URL_BASE='http://www.wrc.com'

In [9]:
from bs4 import BeautifulSoup 
import requests

def getRalliesList(url='http://www.wrc.com/en/wrc/results/rally-results/page/3049----.html'):
    html=requests.get(url)
    soup=BeautifulSoup(html.content, "html5lib")
    #BeautifulSoup has a routine - find_all() - that will find all the HTML tags of a particular sort
    #Links are represented in HTML pages in the form <a href="http//example.com/page.html">link text</a>
    #Grab all the <a> (anchor) tags...
    souptable=soup.findAll("ul")[12].findAll('a')
    rallies={}
    for a in souptable:
        print(a['href'],a.find('img')['title'])#(a['title'],a['href'])
        rallies[a['href'].replace('/en/wrc/results/','').split('/')[0]] = {'path':a['href'],
                                                                           'name':a.find('img')['title']}
    return rallies

In [8]:
rallies = getRalliesList()

/en/wrc/results/monte-carlo/stage-times/page/318-226---.html Rallye Monte-Carlo
/en/wrc/results/sweden/stage-times/page/326-227---.html Rally Sweden
/en/wrc/results/mexico/stage-times/page/334-228---.html Rally Guanajuato México
/en/wrc/results/france/stage-times/page/400-236---.html Che Guevara Energy Drink Tour de Corse
/en/wrc/results/argentina/stage-times/page/346-230---.html YPF Rally Argentina
/en/wrc/results/portugal/stage-times/page/342-229---.html Vodafone Rally de Portugal
/en/wrc/results/italy/stage-times/page/360-231---.html Rally Italia Sardegna
/en/wrc/results/poland/stage-times/page/368-232---.html ORLEN 74th Rally Poland
/en/wrc/results/finland/stage-times/page/376-233---.html Neste Rally Finland
/en/wrc/results/germany/stage-times/page/384-234---.html ADAC Rallye Deutschland
/en/wrc/results/spain/stage-times/page/408-237---.html RallyRACC Catalunya - Rally de España
/en/wrc/results/wales/stage-times/page/416-238---.html Dayinsure Wales Rally GB 
/en/wrc/results/austral

In [ ]:
rally = 'italy'

In [9]:
def getRallyDetails(rallies, rally):
    #url='http://www.wrc.com/en/wrc/results/monte-carlo/stage-times/page/318-226---.html'
    url='{base}{path}'.format(base=URL_BASE,path=rallies[rally]['path'])

    html=requests.get(url)
    soup=BeautifulSoup(html.content, "html5lib")
    #BeautifulSoup has a routine - find_all() - that will find all the HTML tags of a particular sort
    #Links are represented in HTML pages in the form <a href="http//example.com/page.html">link text</a>
    #Grab all the <a> (anchor) tags...
    souptable=soup.find("div",{'class':'content_full frontpage-livecenter'}).findAll('a')
    #souptable
    lookup={}
    for a in souptable:
        lookup[''.join(a.text.strip().split())] = a['href']
    return lookup

lookup = getRallyDetails(rallies, rally)
lookup

{'ChampionshipStandings': '/live-ticker/daten/2017/matrix_driver.all.html',
 'Itinerary': '/live-ticker/daten/2017/228/itinerary.228.all.html',
 'Overall': '/live-ticker/daten/2017/228/stage.228.0.overall.all.html',
 'Penalties': '/live-ticker/daten/2017/228/penalties.228.all.html',
 'Retirements': '/live-ticker/daten/2017/228/retirements.228.all.html',
 'SplitTimes': '/live-ticker/daten/2017/228/split.228.0.all.html',
 'StageTimes': '/live-ticker/daten/2017/228/stage.228.0.all.html',
 'Stagewinners': '/live-ticker/daten/2017/228/stagewinners.228.all.html',
 'Startlists': '/live-ticker/daten/2017/228/entry.228.all.html'}

In [10]:
def getStageURLs(rallies,rally, optionpath):
    url='{base}{path}'.format(base=URL_BASE,path=optionpath)
    html=requests.get(url)
    soup=BeautifulSoup(html.content, "html5lib")
    souptable=soup.find("select",{'name':'rally_id'}).findAll('option')
    options={}
    for o in souptable:
        options[o.text.replace('\n','').strip()] = o['value']
    return options

In [10]:
def getChampionshipURLs(url='http://www.wrc.com/en/wrc/results/championship-standings/page/4176----.html'):
    html=requests.get(url)
    soup=BeautifulSoup(html.content, "html5lib")
    souptable=soup.find("div",{'class':'content_full frontpage-livecenter'}).findAll('a')
    championships={}
    for a in souptable:
        championships[a.text.strip()] = {'path':a['href']}
    return championships


championship = getChampionshipURLs()
championship

{'Archive': {'path': '/live-ticker/daten/wm_all.html'},
 'Co-Drivers': {'path': '/live-ticker/daten/2017/matrix_codriver.html'},
 'Drivers': {'path': '/live-ticker/daten/2017/matrix_driver.html'},
 'Teams': {'path': '/live-ticker/daten/2017/matrix_teams.html'}}

In [11]:
def getStartinglistURLs(rallies,rally, optionpath):
    lookupstarts = getStageURLs(rallies,rally, optionpath)
    lookupstarts = {k.replace('Start List for ',''):lookupstarts[k] for k in lookupstarts}
    return lookupstarts

lookupstarts = getStartinglistURLs(rallies,rally, lookup['Startlists'] )
lookupstarts

{'Entry List': '/live-ticker/daten/2017/228/entry.228.all.html',
 'Section 1 - TC0': '/live-ticker/daten/2017/228/startinglist.228.764.all.html',
 'Section 2 - TC1B': '/live-ticker/daten/2017/228/startinglist.228.766.all.html',
 'Section 4 - TC9E': '/live-ticker/daten/2017/228/startinglist.228.768.all.html',
 'Section 6 - TC15D': '/live-ticker/daten/2017/228/startinglist.228.770.all.html'}

In [12]:
def getOverallURLs(rallies,rally, optionpath):
    overall = getStageURLs(rallies,rally, optionpath)
    return overall

overall = getOverallURLs(rallies,rally, lookup['Overall'] )
overall

{'Final Results': '/live-ticker/daten/2017/228/finalresult.overall.228.all.html',
 'SS1': '/live-ticker/daten/2017/228/stage.228.1.overall.all.html',
 'SS10': '/live-ticker/daten/2017/228/stage.228.10.overall.all.html',
 'SS11': '/live-ticker/daten/2017/228/stage.228.11.overall.all.html',
 'SS12': '/live-ticker/daten/2017/228/stage.228.12.overall.all.html',
 'SS13': '/live-ticker/daten/2017/228/stage.228.13.overall.all.html',
 'SS14': '/live-ticker/daten/2017/228/stage.228.14.overall.all.html',
 'SS15': '/live-ticker/daten/2017/228/stage.228.15.overall.all.html',
 'SS16': '/live-ticker/daten/2017/228/stage.228.16.overall.all.html',
 'SS17': '/live-ticker/daten/2017/228/stage.228.17.overall.all.html',
 'SS18': '/live-ticker/daten/2017/228/stage.228.18.overall.all.html',
 'SS19': '/live-ticker/daten/2017/228/stage.228.19.overall.all.html',
 'SS2': '/live-ticker/daten/2017/228/stage.228.2.overall.all.html',
 'SS3': '/live-ticker/daten/2017/228/stage.228.3.overall.all.html',
 'SS4': '/live

In [13]:
def getSplitTimesURLs(rallies,rally, optionpath):
    overall = getStageURLs(rallies,rally, optionpath)
    return overall

splittimes = getOverallURLs(rallies,rally, lookup['SplitTimes'] )
splittimes

{'SS1': '/live-ticker/daten/2017/228/split.228.1.all.html',
 'SS10': '/live-ticker/daten/2017/228/split.228.10.all.html',
 'SS11': '/live-ticker/daten/2017/228/split.228.11.all.html',
 'SS12': '/live-ticker/daten/2017/228/split.228.12.all.html',
 'SS13': '/live-ticker/daten/2017/228/split.228.13.all.html',
 'SS14': '/live-ticker/daten/2017/228/split.228.14.all.html',
 'SS15': '/live-ticker/daten/2017/228/split.228.15.all.html',
 'SS16': '/live-ticker/daten/2017/228/split.228.16.all.html',
 'SS17': '/live-ticker/daten/2017/228/split.228.17.all.html',
 'SS18': '/live-ticker/daten/2017/228/split.228.18.all.html',
 'SS19': '/live-ticker/daten/2017/228/split.228.19.all.html',
 'SS2': '/live-ticker/daten/2017/228/split.228.2.all.html',
 'SS3': '/live-ticker/daten/2017/228/split.228.3.all.html',
 'SS4': '/live-ticker/daten/2017/228/split.228.4.all.html',
 'SS5': '/live-ticker/daten/2017/228/split.228.5.all.html',
 'SS6': '/live-ticker/daten/2017/228/split.228.6.all.html',
 'SS7': '/live-ticke

In [14]:
def getStageTimesURLs(rallies,rally, optionpath):
    overall = getStageURLs(rallies,rally, optionpath)
    return overall

stagetimes = getOverallURLs(rallies,rally, lookup['StageTimes'] )
stagetimes

{'SS1': '/live-ticker/daten/2017/228/stage.228.1.all.html',
 'SS10': '/live-ticker/daten/2017/228/stage.228.10.all.html',
 'SS11': '/live-ticker/daten/2017/228/stage.228.11.all.html',
 'SS12': '/live-ticker/daten/2017/228/stage.228.12.all.html',
 'SS13': '/live-ticker/daten/2017/228/stage.228.13.all.html',
 'SS14': '/live-ticker/daten/2017/228/stage.228.14.all.html',
 'SS15': '/live-ticker/daten/2017/228/stage.228.15.all.html',
 'SS16': '/live-ticker/daten/2017/228/stage.228.16.all.html',
 'SS17': '/live-ticker/daten/2017/228/stage.228.17.all.html',
 'SS18': '/live-ticker/daten/2017/228/stage.228.18.all.html',
 'SS19': '/live-ticker/daten/2017/228/stage.228.19.all.html',
 'SS2': '/live-ticker/daten/2017/228/stage.228.2.all.html',
 'SS3': '/live-ticker/daten/2017/228/stage.228.3.all.html',
 'SS4': '/live-ticker/daten/2017/228/stage.228.4.all.html',
 'SS5': '/live-ticker/daten/2017/228/stage.228.5.all.html',
 'SS6': '/live-ticker/daten/2017/228/stage.228.6.all.html',
 'SS7': '/live-ticke

In [15]:
def getStagewinnersURLs(rallies,rally, optionpath):
    overall = getStageURLs(rallies,rally, optionpath)
    return overall

stagewinners = getOverallURLs(rallies,rally, lookup['Stagewinners'] )
stagewinners

{'Day 1': '/live-ticker/daten/2017/228/day.228.1.all.html',
 'Day 2': '/live-ticker/daten/2017/228/day.228.2.all.html',
 'Day 3': '/live-ticker/daten/2017/228/day.228.3.all.html',
 'Day 4': '/live-ticker/daten/2017/228/day.228.4.all.html',
 'Final Results': '/live-ticker/daten/2017/228/finalresult.228.all.html',
 'Stagewinners': '/live-ticker/daten/2017/228/stagewinners.228.all.html'}

In [16]:
def _getBaseItinerary(path):
    url='{base}{path}'.format(base=URL_BASE, path=path)
    df_itinerary=loader(url)[0]
    if not validateLoadedDF(df_itinerary): return DataFrame()
    
    df_itinerary.columns=['stage','stagename','distance (km)','firstCar','status']
    return df_itinerary

def getItinerary(path):
    df_itinerary = _getBaseItinerary(path)
    if not validateLoadedDF(df_itinerary): return DataFrame()

    df_itinerary['stagename']=df_itinerary['stagename'].str.extract(tagcontent2RE_HTML,expand=True)[1]
    df_itinerary['stage']=df_itinerary['stage'].str.extract(tagcontent2RE_HTML,expand=True)[1]
    df_itinerary['stageNo']=df_itinerary['stage'].str.replace('SS','')
    df_itinerary['basestage']=df_itinerary['stage'].where(df_itinerary['stage'].str.startswith('SS'),
                                                          'SS'+df_itinerary['stage'].str.extract(stagenumRE,
                                                                                                 expand=True)[0])

    #Add the day number
    df_itinerary['day']=df_itinerary['stage'].str.startswith('Day')
    df_itinerary['day']=df_itinerary['day'].fillna(False).cumsum()    

    #Add the section
    df_itinerary['section']=df_itinerary['stage'].str.contains(r'TC[0-9]+B')
    df_itinerary['section']=1+df_itinerary['section'].fillna(False).cumsum()

    return df_itinerary

In [17]:
!rm 'test1.db'
dbname='test1.db'
conn = sqlite3.connect(dbname)

In [18]:
table='Itinerary'
YEAR=2017

df=getItinerary(lookup[table])

df['Year']=YEAR
df['Rally']=rally

df.to_sql(table.lower(), conn, index=False, if_exists='append')

/usr/local/lib/python3.6/site-packages/pandas/core/generic.py:1534: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [19]:
df

,stage,stagename,distance (km),firstCar,status,stageNo,basestage,day,section,Year,Rally
0,Day 1 - 08.06.2017,NaN,NaN,NaN,NaN,Day 1 - 08.06.2017,NaN,1,1,2017,italy
1,SS1,ITTIRI ARENA SHOW,2.00,18:00,COMPLETED,1,SS1,1,1,2017,italy
2,Day 2 - 09.06.2017,NaN,NaN,NaN,NaN,Day 2 - 09.06.2017,NaN,2,1,2017,italy
3,SS2,TERRANOVA 1,14.54,07:18,COMPLETED,2,SS2,2,1,2017,italy
4,SS3,MONTE OLIA 1,19.05,07:41,COMPLETED,3,SS3,2,1,2017,italy
5,SS4,TULA 1,15.00,09:12,COMPLETED,4,SS4,2,1,2017,italy
6,SS5,TERGU-OSILO 1,14.14,10:17,COMPLETED,5,SS5,2,1,2017,italy
7,TC5B,Regroup &amp; Technical Zone OUT / Service IN,,11:56,COMPLETED,TC5B,SS5,2,2,2017,italy
8,SS6,TERRANOVA 2,14.54,15:25,COMPLETED,6,SS6,2,2,2017,italy
9,SS7,MONTE OLIA 2,19.05,15:48,COMPLETED,7,SS7,2,2,2017,italy


In [20]:
q="SELECT name FROM sqlite_master WHERE type='table';"

pd.read_sql_query(q,conn)

,name
0,itinerary


In [21]:
q="SELECT * FROM itinerary LIMIT 3;"

pd.read_sql_query(q,conn)

,stage,stagename,distance (km),firstCar,status,stageNo,basestage,day,section,Year,Rally
0,Day 1 - 08.06.2017,None,None,None,None,Day 1 - 08.06.2017,None,1,1,2017,italy
1,SS1,ITTIRI ARENA SHOW,2.00,18:00,COMPLETED,1,SS1,1,1,2017,italy
2,Day 2 - 09.06.2017,None,None,None,None,Day 2 - 09.06.2017,None,2,1,2017,italy


In [72]:
championship

{'Archive': {'path': '/live-ticker/daten/wm_all.html'},
 'Co-Drivers': {'path': '/live-ticker/daten/2017/matrix_codriver.html'},
 'Drivers': {'path': '/live-ticker/daten/2017/matrix_driver.html'},
 'Teams': {'path': '/live-ticker/daten/2017/matrix_teams.html'}}

In [73]:
def _getChampionshipStandingsBase(path, typ='Drivers'):
    url='{base}{path}'.format(base=URL_BASE, path=path)
    df_standings=loader(url)[0]
    if not validateLoadedDF(df_standings): return DataFrame()
    cols = ['pos','name']
    #rallynames is a list of the rally names
    rallynames = df_standings.iloc[0,2:-1].str.extract(imgAltRE_HTML, expand=True)[0].apply(lambda x: unicodify(x)).tolist()
    rallycols = ['Round_{}'.format(i) for i in range(1,1+len(rallynames))]
    cols = cols + rallycols
    cols=cols+['Total']
    df_standings.columns=cols
    df_standings['pos']=to_numeric(df_standings['pos'].str.strip('.'), errors='coerce')
    df_standings['Total'] = to_numeric(df_standings['Total'], errors='coerce')
    
    if typ!='Teams':
        df_standings['name'],df_standings['tmp']=_driverNameNationality(df_standings,'name')
        df_standings = df_standings.drop(columns = ['tmp'])
    df_standings = df_standings.drop([0])
    return df_standings, rallynames

def _ppx(x):
    points=None
    bonus=None
    if isinstance(x,list):
        if len(x)==2:
            points  = x[0]
            bonus = x[1]
        elif len(x)==1 and x[0]!='':
            points=x[0]
    return points, bonus

def getChampionshipStandings(path, typ='Drivers'):
    df_standings, rallynames=_getChampionshipStandingsBase(path, typ=typ)
    if not validateLoadedDF(df_standings): return DataFrame()
    #return df_standings
    for c in [col for col in df_standings if col.startswith('Round')]:
        #Get the points
        tmp=df_standings[c].str.replace('\n','').str.extract(standing_pointsRE, expand=True)
        if typ != 'Teams':
            tmp[1] = to_numeric(tmp[1].str.strip('.'), errors='coerce')
            df_standings['pos{}'.format(c)] = tmp[1]
            tmp2=tmp[0].str.split('+').apply(lambda x:_ppx(x)).apply(Series)
            for c2 in [0,1]:
                tmp[c2] = to_numeric(tmp2[c2], errors='coerce')
            df_standings['points{}'.format(c)]=tmp[0]
            df_standings['bonus{}'.format(c)]=tmp[1]
        else:
            tmp2 = tmp[1].str.split('+').apply(lambda x:_ppx(x)).apply(Series)
            for c2 in [0,1]:
                tmp2[c2] = to_numeric(tmp2[c2], errors='coerce')
            df_standings['D1pos{}'.format(c)] = tmp2[0]
            df_standings['D2pos{}'.format(c)] = tmp2[1]
            tmp2=tmp[0].str.split('+').apply(lambda x:_ppx(x)).apply(Series)
            for c2 in [0,1]:
                tmp[c2] = to_numeric(tmp2[c2], errors='coerce')
            df_standings['D1points{}'.format(c)]=tmp2[0]
            df_standings['D2points{}'.format(c)]=tmp2[1]

    if typ != 'Teams':
        df_standings =  wide_to_long(df_standings, ["Round_","bonusRound_", "posRound_","pointsRound_"],
                                    i=["pos",'name'], j="roundnum").reset_index()
    else:
        df_standings =  wide_to_long(df_standings, ["Round_","D1posRound_", "D2posRound_",
                                                    "D1pointsRound_", "D2pointsRound_"],
                                    i=["pos",'name'], j="roundnum").reset_index()


    df_standings['rallyname'] = df_standings['roundnum'].apply(lambda x: rallynames[int(x)-1] )
    df_standings.columns = [ c. strip('_') for c in df_standings.columns]
    return df_standings

typ='Teams'
df=getChampionshipStandings(championship[typ]['path'], typ=typ)
df.head(10)

,pos,name,roundnum,Total,Round,D1posRound,D2posRound,D1pointsRound,D2pointsRound,rallyname
0,1.0,M-Sport World Rally Team,1,428.0,"25+15\n<span class=""newline"">\n ...",1.0,3.0,25,15,85e Rallye Monte-Carlo 2017
1,1.0,M-Sport World Rally Team,2,428.0,"18+15\n<span class=""newline"">\n ...",2.0,3.0,18,15,Rally Sweden
2,1.0,M-Sport World Rally Team,3,428.0,"18+12\n<span class=""newline"">\n ...",2.0,4.0,18,12,Rally Guanajuato México
3,1.0,M-Sport World Rally Team,4,428.0,"18+8\n<span class=""newline"">\n ...",2.0,11.0,18,8,Tour de Corse 2017
4,1.0,M-Sport World Rally Team,5,428.0,"18+15\n<span class=""newline"">\n ...",2.0,3.0,18,15,Rally Argentina
5,1.0,M-Sport World Rally Team,6,428.0,"25+12\n<span class=""newline"">\n ...",1.0,4.0,25,12,Vodafone Rally de Portugal
6,1.0,M-Sport World Rally Team,7,428.0,"25+10\n<span class=""newline"">\n ...",1.0,5.0,25,10,Rally Italia Sardegna
7,1.0,M-Sport World Rally Team,8,428.0,"15+10\n<span class=""newline"">\n ...",3.0,8.0,15,10,ORLEN 74th Rally Poland
8,1.0,M-Sport World Rally Team,9,428.0,"18+8\n<span class=""newline"">\n ...",2.0,7.0,18,8,Neste Rally Finland
9,1.0,M-Sport World Rally Team,10,428.0,"25+15\n<span class=""newline"">\n ...",1.0,3.0,25,15,ADAC Rally Deutschland


In [63]:
df['Round_1'].iloc[2]

'18+6\n<span class="newline">\n                  (2+16)\n                  </span>'

In [32]:
rallynames = ['85e Rallye Monte-Carlo 2017', 'Rally Sweden', 'Rally Guanajuato México', 'Tour de Corse 2017', 'Rally Argentina', 'Vodafone Rally de Portugal', 'Rally Italia Sardegna', 'ORLEN 74th Rally Poland', 'Neste Rally Finland', 'ADAC Rally Deutschland', 'RallyRACC-Catalunya-Costa Daurada', 'Wales Rally GB', 'Kennards Hire Rally Australia']

In [33]:
dfs =  wide_to_long(df, ["Round_","bonusRound_", "posRound_","pointsRound_"],
                                        i=["pos",'name'], j="roundnum").reset_index()
dfs['rallyname'] = dfs['roundnum'].apply(lambda x: rallynames[int(x)-1] )
dfs.head()

,pos,name,roundnum,Total,Round_,bonusRound_,posRound_,pointsRound_,rallyname
0,1.0,M-Sport World Rally Team,1,428,"25+15\n<span class=""newline"">\n ...",15,NaN,25,85e Rallye Monte-Carlo 2017
1,1.0,M-Sport World Rally Team,2,428,"18+15\n<span class=""newline"">\n ...",15,NaN,18,Rally Sweden
2,1.0,M-Sport World Rally Team,3,428,"18+12\n<span class=""newline"">\n ...",12,NaN,18,Rally Guanajuato México
3,1.0,M-Sport World Rally Team,4,428,"18+8\n<span class=""newline"">\n ...",8,NaN,18,Tour de Corse 2017
4,1.0,M-Sport World Rally Team,5,428,"18+15\n<span class=""newline"">\n ...",15,NaN,18,Rally Argentina


In [22]:
def _getPenaltiesBase(path):
    url='{base}{path}'.format(base=URL_BASE, path=path)
    df_penalties=loader(url)[0]
    if not validateLoadedDF(df_penalties): return DataFrame()
    
    df_penalties.columns=['carNo', 'driverName', 'coDriverName', 'team_Car', 
                       'eligibility', 'groupClass', 'control', 'reason', 'penalty']
    df_penalties['carNo']=df_penalties['carNo'].astype(str)
    return df_penalties


def getPenalties(path):
    df_penalties=_getPenaltiesBase(path)
    if not validateLoadedDF(df_penalties): return DataFrame()

    for r in ['driverName','coDriverName']:
        df_penalties[r],df_penalties[r+'_nationality']=_driverNameNationality(df_penalties,r)

    df_penalties['team'],df_penalties['car']=_getTeamCar(df_penalties)

    df_penalties['tmp']=df_penalties.apply(lambda x: [y for y in zip(x['reason'].split('\n'),
                                                             x['penalty'].split('\n'),
                                                             x['control'].split('\n'))], axis=1)

    df_tmp = df_penalties.set_index('carNo')['tmp'].apply(Series) # break into separate columns
    df_tmp = df_tmp.stack().reset_index('carNo')
    df_tmp.rename(columns={0: 'penreasoncontrol'}, inplace=True)

    df_penalties=merge(df_penalties,df_tmp,on='carNo')
    df_penalties[['reason2','penalty2','control2']]=df_penalties['penreasoncontrol'].apply(Series)
    df_penalties['td_penalty']=df_penalties.apply(lambda x: regularTimeString(x['penalty2']),axis=1)
    df_penalties['eligibility']=df_penalties['eligibility']

    df_penalties['stagenumber']=df_penalties['control2'].str.extract(controlRE, expand=True).astype(int)
    df_penalties.drop(['tmp', 'penreasoncontrol'], axis=1, inplace=True)
    return df_penalties
    

In [23]:
table='Penalties'
df=getPenalties(lookup[table])

df['Year']=YEAR
df['Rally']=rally

df.to_sql(table.lower(), conn, index=False, if_exists='append')

q="SELECT * FROM {}  LIMIT 3;".format(table)

pd.read_sql_query(q,conn)

/usr/local/lib/python3.6/site-packages/pandas/io/sql.py:473: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  chunksize=chunksize, dtype=dtype)


,carNo,driverName,coDriverName,team_Car,eligibility,groupClass,control,reason,penalty,driverName_nationality,coDriverName_nationality,team,car,reason2,penalty2,control2,td_penalty,stagenumber,Year,Rally
0,6,D. SORDO,M. MARTI,HYUNDAI MOTORSPORT\nHyundai I20 COUPE WRC,M,RC1,TC5A\nTC5C\nTC7,4M LATE\n5M LATE\n2M LATE,00:40\n00:50\n00:20,ESP,ESP,HYUNDAI MOTORSPORT,Hyundai I20 COUPE WRC,4M LATE,00:40,TC5A,40000000000,5,2017,italy
1,6,D. SORDO,M. MARTI,HYUNDAI MOTORSPORT\nHyundai I20 COUPE WRC,M,RC1,TC5A\nTC5C\nTC7,4M LATE\n5M LATE\n2M LATE,00:40\n00:50\n00:20,ESP,ESP,HYUNDAI MOTORSPORT,Hyundai I20 COUPE WRC,5M LATE,00:50,TC5C,50000000000,5,2017,italy
2,6,D. SORDO,M. MARTI,HYUNDAI MOTORSPORT\nHyundai I20 COUPE WRC,M,RC1,TC5A\nTC5C\nTC7,4M LATE\n5M LATE\n2M LATE,00:40\n00:50\n00:20,ESP,ESP,HYUNDAI MOTORSPORT,Hyundai I20 COUPE WRC,2M LATE,00:20,TC7,20000000000,7,2017,italy


In [24]:
def _getRetirementsBase(path):
    #http://www.wrc.com/live-ticker/daten/2017/216/retirements.216.all.html
    url='{base}{path}'.format(base=URL_BASE, path=path)
    df_retirements=loader(url)[0]
    if not validateLoadedDF(df_retirements): return DataFrame()
    
    df_retirements.columns=['carNo','driverName','coDriverName','team_Car','eligibility','groupClass','control','reason']
    df_retirements['carNo']=df_retirements['carNo'].astype(str)
    return df_retirements

def getRetirements(path):
    df_retirements=_getRetirementsBase(path)
    if not validateLoadedDF(df_retirements): return DataFrame()

    df_retirements['team'],df_retirements['car']=_getTeamCar(df_retirements)

    for r in ['driverName','coDriverName']:
        df_retirements[r],df_retirements[r+'_nationality']=_driverNameNationality(df_retirements,r)

    df_retirements['eligibility']=df_retirements['eligibility']

    df_retirements['stagenumber']=df_retirements['control'].str.extract(controlRE, expand=True).astype(int)

    return df_retirements

In [25]:
table='Retirements'
df=getRetirements(lookup[table])

df['Year']=YEAR
df['Rally']=rally

df.to_sql(table.lower(), conn, index=False, if_exists='append')

q="SELECT * FROM {}  LIMIT 3;".format(table)

pd.read_sql_query(q,conn)

,carNo,driverName,coDriverName,team_Car,eligibility,groupClass,control,reason,team,car,driverName_nationality,coDriverName_nationality,stagenumber,Year,Rally
0,3,E. EVANS,D. BARRITT,M-SPORT WORLD RALLY TEAM\nFord Fiesta WRC,M,RC1,TC19C,,M-SPORT WORLD RALLY TEAM,Ford Fiesta WRC,GBR,GBR,19,2017,italy
1,4,H. PADDON,S. MARSHALL,HYUNDAI MOTORSPORT\nHyundai I20 COUPE WRC,M,RC1,SF18,,HYUNDAI MOTORSPORT,Hyundai I20 COUPE WRC,NZL,GBR,18,2017,italy
2,7,K. MEEKE,P. NAGLE,Citroën Total Abu Dhabi WRT\nCitroen C3 WRC,M,RC1,TC5C,,Citroën Total Abu Dhabi WRT,Citroen C3 WRC,GBR,IRL,5,2017,italy


In [26]:
def _getEntryListBase(path):
    url='{base}{path}'.format(base=URL_BASE, path=path)
    df_entry=loader(url)[0]
    if not validateLoadedDF(df_entry): return DataFrame()
    
    cols=['carNo','driver_Codriver','team_Car','eligibility','groupClass','FIApriority']
    df_entry.columns=cols
    df_entry.fillna(0,inplace=True)
    for j in cols:
        df_entry[j]=df_entry[j].astype(str)
    #Make null competition entries None
    df_entry.loc[df_entry['eligibility']=='0','eligibility']=None
    return df_entry

def getEntryList(path):
    df_entry=_getEntryListBase(path)
    if not validateLoadedDF(df_entry): return DataFrame()

    df_entry['team'], df_entry['car'] = _getTeamCar(df_entry)

    df_entry['driverName'],df_entry['driverName_nationality'], \
              df_entry['coDriverName'],df_entry['coDriverName_nationality'] = _driverCodriverNameNationality(df_entry)

    return df_entry


In [27]:
table='entrylist'
df=getEntryList(lookup['Startlists'])

df['Year']=YEAR
df['Rally']=rally

df.to_sql(table, conn, index=False, if_exists='append')

q="SELECT * FROM {}  LIMIT 3;".format(table)

pd.read_sql_query(q,conn)

,carNo,driver_Codriver,team_Car,eligibility,groupClass,FIApriority,team,car,driverName,coDriverName,driverName_nationality,coDriverName_nationality,Year,Rally
0,1,"<img src=""http://logo.wrc.com/flaggen/3FRA_18x...",M-SPORT WORLD RALLY TEAM\nFord Fiesta WRC,M,RC1,P1,M-SPORT WORLD RALLY TEAM,Ford Fiesta WRC,S. OGIER,J. INGRASSIA,FRA,FRA,2017,italy
1,2,"<img src=""http://logo.wrc.com/flaggen/3EST_18x...",M-SPORT WORLD RALLY TEAM\nFord Fiesta WRC,M,RC1,P1,M-SPORT WORLD RALLY TEAM,Ford Fiesta WRC,O. TANAK,M. JARVEOJA,EST,EST,2017,italy
2,3,"<img src=""http://logo.wrc.com/flaggen/3GBR_18x...",M-SPORT WORLD RALLY TEAM\nFord Fiesta WRC,M,RC1,P1,M-SPORT WORLD RALLY TEAM,Ford Fiesta WRC,E. EVANS,D. BARRITT,GBR,GBR,2017,italy


In [28]:
def _getStartListBase(path):
    url='{base}{path}'.format(base=URL_BASE, path=path)
    df_starts=loader(url)[0]
    if not validateLoadedDF(df_starts): return DataFrame()
    
    cols=['carNo','driver_Codriver','team_Car','eligibility','startTime']
    df_starts.columns=cols
    df_starts.fillna(0,inplace=True)
    for j in cols:
        df_starts[j]=df_starts[j].astype(str)
    #Make null competition entries None
    df_starts.loc[df_starts['eligibility']=='0','eligibility']=None
    return df_starts

def getStartList(path):
    df_starts=_getStartListBase(path)
    if not validateLoadedDF(df_starts): return DataFrame()
    df_starts['team'], df_starts['car'] = _getTeamCar(df_starts)

    df_starts['driverName'],df_starts['driverName_nationality'], \
              df_starts['coDriverName'],df_starts['coDriverName_nationality'] = _driverCodriverNameNationality(df_starts)

    return df_starts

In [29]:
table='startlists'
for s in lookupstarts:
    if s != 'Entry List':
        df=getStartList(lookupstarts[s])
        df['section'] = s
        df['Year']=YEAR
        df['Rally']=rally

        df.to_sql(table, conn, index=False, if_exists='append')
        
q="SELECT * FROM {}  LIMIT 3;".format(table)

pd.read_sql_query(q,conn)

,carNo,driver_Codriver,team_Car,eligibility,startTime,team,car,driverName,coDriverName,driverName_nationality,coDriverName_nationality,section,Year,Rally
0,98,"<img src=""http://logo.wrc.com/flaggen/3ITA_18x...",SAMUELE PIROTTO\nAbarth 500 R3T,,17:00,SAMUELE PIROTTO,Abarth 500 R3T,S. PIROTTO,E. REVELLO,ITA,ITA,Section 1 - TC0,2017,italy
1,97,"<img src=""http://logo.wrc.com/flaggen/3ITA_18x...",ANDREA COTI ZELATI\nAbarth 500 R3T,,17:01,ANDREA COTI ZELATI,Abarth 500 R3T,A. COTI ZELATI,L. ALIBERTO,ITA,ITA,Section 1 - TC0,2017,italy
2,40,"<img src=""http://logo.wrc.com/flaggen/3ITA_18x...",ACI TEAM ITALIA\nHyundai I20 R5,WRC2,17:05,ACI TEAM ITALIA,Hyundai I20 R5,F. ANDOLFI,M. FENOLI,ITA,ITA,Section 1 - TC0,2017,italy


In [30]:
def _getStagewinnersBase(path):
    url='{base}{path}'.format(base=URL_BASE, path=path)
    df_stagewinners=loader(url)[0]
    if not validateLoadedDF(df_stagewinners): return DataFrame()
    cols=['stage','stagename','driverName','team','winnerTime']#'carNo','driver_Codriver','team_Car','eligibility','groupClass','FIApriority']
    df_stagewinners.columns=cols
    df_stagewinners['driverName'] = df_stagewinners['driverName'].str.replace('\n','')
    df_stagewinners['driverName'],df_stagewinners['driverName_nationality']=_driverNameNationality(df_stagewinners,'driverName')
    return df_stagewinners
    
def getStagewinners(path):
    df_stagewinners=_getStagewinnersBase(path)
    if not validateLoadedDF(df_stagewinners): return DataFrame()
    
    return df_stagewinners

In [31]:
#Seems we only have results from stagewinners['Stagewinners'], which is the default from lookup ?

table='Stagewinners'
df=getStagewinners(lookup[table])

df['Year']=YEAR
df['Rally']=rally

df
#df.to_sql(table.lower(), conn, index=False, if_exists='append')

#q="SELECT * FROM {}  LIMIT 3;".format(table)

#pd.read_sql_query(q,conn)

,stage,stagename,driverName,team,winnerTime,driverName_nationality,Year,Rally
0,SS1,ITTIRI ARENA SHOW (2.00 km),N. GILSOUL,HYUNDAI MOTORSPORT,2:01.8,BEL,2017,italy
1,SS2,TERRANOVA 1 (14.54 km),P. NAGLE,Citroën Total Abu Dhabi WRT,10:14.1,IRL,2017,italy
2,SS3,MONTE OLIA 1 (19.05 km),K. LINDSTROM,TOYOTA GAZOO RACING WRT,13:25.8,FIN,2017,italy
3,SS4,TULA 1 (15.00 km),M. MARTI,HYUNDAI MOTORSPORT,11:51.2,ESP,2017,italy
4,SS5,TERGU-OSILO 1 (14.14 km),J. FERM,TOYOTA GAZOO RACING WRT,9:06.3,FIN,2017,italy
5,SS6,TERRANOVA 2 (14.54 km),J. FERM,TOYOTA GAZOO RACING WRT,9:59.5,FIN,2017,italy
6,SS7,MONTE OLIA 2 (19.05 km),J. FERM,TOYOTA GAZOO RACING WRT,13:04.9,FIN,2017,italy
7,SS8,TULA 2 (15.00 km),M. MARTI,HYUNDAI MOTORSPORT,11:38.5,ESP,2017,italy
8,SS9,TERGU-OSILO 2 (14.14 km),M. MARTI,HYUNDAI MOTORSPORT,8:53.2,ESP,2017,italy
9,SS10,COILUNA - LOELLE 1 (14.95 km),S. MARSHALL,HYUNDAI MOTORSPORT,8:00.8,GBR,2017,italy


In [32]:
#Overall
def _getOverallBase(path):
    url='{base}{path}'.format(base=URL_BASE, path=path)
    df_overall=loader(url)[0]
    if not validateLoadedDF(df_overall): return DataFrame()
    
    cols=['pos', 'carNo', 'driverName', 'coDriverName', 'team',
          'eligibility', 'groupClass', 'time', 'diffPrev', 'diffFirst']
    df_overall.columns=cols
    df_overall.fillna(0,inplace=True)
    df_overall['pos']=df_overall['pos'].astype(float)
    cols.remove('pos')
    for j in cols:
        df_overall[j]=df_overall[j].astype(str)
    for r in ['driverName','coDriverName']:
        df_overall[r],df_overall[r+'_nationality']=_driverNameNationality(df_overall,r)

    return df_overall#.reset_index(drop=True)

def getOverall(path):
    df_overall=_getOverallBase(path)
    if not validateLoadedDF(df_overall): return DataFrame()
    
    for col in ['time','diffPrev','diffFirst']:
        df_overall['td_'+col]=df_overall.apply(lambda x: regularTimeString(x[col]),axis=1)

    return df_overall

In [33]:
overall

{'Final Results': '/live-ticker/daten/2017/228/finalresult.overall.228.all.html',
 'SS1': '/live-ticker/daten/2017/228/stage.228.1.overall.all.html',
 'SS10': '/live-ticker/daten/2017/228/stage.228.10.overall.all.html',
 'SS11': '/live-ticker/daten/2017/228/stage.228.11.overall.all.html',
 'SS12': '/live-ticker/daten/2017/228/stage.228.12.overall.all.html',
 'SS13': '/live-ticker/daten/2017/228/stage.228.13.overall.all.html',
 'SS14': '/live-ticker/daten/2017/228/stage.228.14.overall.all.html',
 'SS15': '/live-ticker/daten/2017/228/stage.228.15.overall.all.html',
 'SS16': '/live-ticker/daten/2017/228/stage.228.16.overall.all.html',
 'SS17': '/live-ticker/daten/2017/228/stage.228.17.overall.all.html',
 'SS18': '/live-ticker/daten/2017/228/stage.228.18.overall.all.html',
 'SS19': '/live-ticker/daten/2017/228/stage.228.19.overall.all.html',
 'SS2': '/live-ticker/daten/2017/228/stage.228.2.overall.all.html',
 'SS3': '/live-ticker/daten/2017/228/stage.228.3.overall.all.html',
 'SS4': '/live

In [214]:
getOverall(overall['SS1']).head()

,pos,carNo,driverName,coDriverName,team,eligibility,groupClass,time,diffPrev,diffFirst,driverName_nationality,coDriverName_nationality,td_time,td_diffPrev,td_diffFirst
0,1.0,5,T. NEUVILLE,N. GILSOUL,HYUNDAI MOTORSPORT,M,RC1,2:01.8,,,BEL,BEL,00:02:01.800000,NaT,NaT
1,2.0,2,O. TANAK,M. JARVEOJA,M-SPORT WORLD RALLY TEAM,M,RC1,2:02.0,+0.2,+0.2,EST,EST,00:02:02,00:00:00.200000,00:00:00.200000
2,3.0,6,D. SORDO,M. MARTI,HYUNDAI MOTORSPORT,M,RC1,2:02.2,+0.2,+0.4,ESP,ESP,00:02:02.200000,00:00:00.200000,00:00:00.400000
3,4.0,3,E. EVANS,D. BARRITT,M-SPORT WORLD RALLY TEAM,M,RC1,2:02.3,+0.1,+0.5,GBR,GBR,00:02:02.300000,00:00:00.100000,00:00:00.500000
4,5.0,4,H. PADDON,S. MARSHALL,HYUNDAI MOTORSPORT,M,RC1,2:02.7,+0.4,+0.9,NZL,GBR,00:02:02.700000,00:00:00.400000,00:00:00.900000


In [35]:
def _getOverallFinalBase(path):
    url='{base}{path}'.format(base=URL_BASE, path=path)
    df_overall=loader(url)[0]
    if not validateLoadedDF(df_overall): return DataFrame()
    cols=['pos','carNo', 'driverName', 'eligibility', 'stageTime', 'penalties', 'totaltime','diffPrev','diffFirst']
    df_overall.columns=cols
    df_overall.fillna(0,inplace=True)
    df_overall['pos']=df_overall['pos'].astype(float)
    cols.remove('pos')
    for j in cols:
        df_overall[j]=df_overall[j].astype(str)
    df_overall['driverName'],df_overall['driverName_nationality']=_driverNameNationality(df_overall,'driverName')

    return df_overall#.reset_index(drop=True)

def getOverallFinal(path):
    df_overall=_getOverallFinalBase(path)
    if not validateLoadedDF(df_overall): return DataFrame()
    
    for col in ['stageTime','totaltime','diffPrev','diffFirst']:
        df_overall['td_'+col]=df_overall.apply(lambda x: regularTimeString(x[col]),axis=1)

    return df_overall

In [36]:
getOverallFinal(overall['Final Results'])

,pos,carNo,driverName,eligibility,stageTime,penalties,totaltime,diffPrev,diffFirst,driverName_nationality,td_stageTime,td_totaltime,td_diffPrev,td_diffFirst
0,1.0,61,N. SOLANS,WRC3JWRC,4:00:07.8,,4:00:07.8,,,ESP,04:00:07.800000,04:00:07.800000,NaT,NaT
1,2.0,62,N. CIAMIN,WRC3JWRC,4:00:43.9,00:10,4:00:53.9,+46.1,+46.1,FRA,04:00:43.900000,04:00:53.900000,00:00:46.100000,00:00:46.100000
2,3.0,64,J. TANNERT,WRC3JWRC,4:03:28.6,,4:03:28.6,+2:34.7,+3:20.8,DEU,04:03:28.600000,04:03:28.600000,00:02:34.700000,00:03:20.800000
3,4.0,63,T. FOLB,WRC3JWRC,4:04:49.0,,4:04:49.0,+1:20.4,+4:41.2,FRA,04:04:49,04:04:49,00:01:20.400000,00:04:41.200000
4,5.0,65,D. RADSTROM,WRC3JWRC,4:05:14.9,01:50,4:07:04.9,+2:15.9,+6:57.1,SWE,04:05:14.900000,04:07:04.900000,00:02:15.900000,00:06:57.100000
5,6.0,66,R. DUGGAN,WRC3JWRC,4:16:19.6,,4:16:19.6,+9:14.7,+16:11.8,IRL,04:16:19.600000,04:16:19.600000,00:09:14.700000,00:16:11.800000
6,7.0,68,D. VAN WAY,WRC3JWRC,4:28:15.4,,4:28:15.4,+11:55.8,+28:07.6,USA,04:28:15.400000,04:28:15.400000,00:11:55.800000,00:28:07.600000
7,8.0,71,E. BRAZZOLI,WRC3,4:59:29.2,,4:59:29.2,+31:13.8,+59:21.4,ITA,04:59:29.200000,04:59:29.200000,00:31:13.800000,00:59:21.400000
8,9.0,67,S. CAREAGA,WRC3JWRC,5:11:47.6,,5:11:47.6,+12:18.4,+1:11:39.8,BOL,05:11:47.600000,05:11:47.600000,00:12:18.400000,01:11:39.800000


In [37]:
#SplitTimes
def _getSplitTimesBase(path):
    url='{base}{path}'.format(base=URL_BASE, path=path)
    df_splits=loader(url)[0]
    if not validateLoadedDF(df_splits): return DataFrame()
    
    #df_splits.fillna(0,inplace=True)

    df_splits.columns=['start','carNo',
                       'driverName','team',
                       'eligibility']+['split_{}'.format(i) for i in range(1,len(df_splits.columns)-5)]+['stageTime']
    #df_splits.fillna(0,inplace=True)
    for j in df_splits.columns:
        if j in ['carNo']:
            df_splits[j]=df_splits[j].astype(str)
        else:
            df_splits['start']=to_numeric(df_splits['start'],downcast='integer')
    return df_splits

def getSplitTimes(path):
    df_splitTimes=_getSplitTimesBase(path)
    if not validateLoadedDF(df_splitTimes): return DataFrame()
    
    df_splitTimes['driverName'],df_splitTimes['driverName_nationality']=_driverNameNationality(df_splitTimes)

    for col in ['stageTime']:
        df_splitTimes['time'+col]=df_splitTimes.apply(lambda x: regularTimeString(x[col]),axis=1)
    df_splitTimes_base=df_splitTimes[:]
    cols=[c for c in df_splitTimes.columns if c.startswith('split_')]
    df_splitTimes_splits=df_splitTimes_base[['carNo','driverName']+cols][:]
    for col in cols:
        df_splitTimes_splits['td'+col]=df_splitTimes_base.apply(lambda x: regularTimeString(x[col]),axis=1)
        df_splitTimes_splits['time'+col]=df_splitTimes_splits['td'+col]
        #This requires that there is a time set in row 0...
        basetime=df_splitTimes_splits.loc[0,'time'+col]
        if not isnull(df_splitTimes_splits.loc[0,'time'+col]):
            df_splitTimes_splits.loc[0,'td'+col]=regularTimeString(0)
            df_splitTimes_splits.loc[0,'time'+col]=regularTimeString(0)
            df_splitTimes_splits['time'+col]=basetime+df_splitTimes_splits['time'+col]
    df_splitTimes_splits = wide_to_long(df_splitTimes_splits, ["split_","tdsplit_", "timesplit_"],
                                        i=["carNo",'driverName'], j="Split").reset_index()
    df_splitTimes_base = df_splitTimes_base[['start', 'carNo', 'driverName', 'team', 'eligibility', 'stageTime', 'driverName_nationality', 'timestageTime']]
    return df_splitTimes_base,df_splitTimes_splits

In [38]:
#splittimes
z,zz=getSplitTimes(splittimes['SS2'])
z.head()

,start,carNo,driverName,team,eligibility,stageTime,driverName_nationality,timestageTime
0,1.0,1,S. OGIER,M-SPORT WORLD RALLY TEAM,M,10:26.1,FRA,00:10:26.100000
1,2.0,5,T. NEUVILLE,HYUNDAI MOTORSPORT,M,10:22.3,BEL,00:10:22.300000
2,3.0,10,J. LATVALA,TOYOTA GAZOO RACING WRT,M,10:24.6,FIN,00:10:24.600000
3,4.0,2,O. TANAK,M-SPORT WORLD RALLY TEAM,M,10:21.9,EST,00:10:21.900000
4,5.0,6,D. SORDO,HYUNDAI MOTORSPORT,M,10:21.5,ESP,00:10:21.500000


In [39]:
zz.head()

,carNo,driverName,Split,split_,tdsplit_,timesplit_
0,1,S. OGIER,1,00:03:28.0,00:00:00,00:03:28
1,1,S. OGIER,2,00:06:48.2,00:00:00,00:06:48.200000
2,5,T. NEUVILLE,1,-1.4,-1 days +23:59:58.600000,00:03:26.600000
3,5,T. NEUVILLE,2,-5.0,-1 days +23:59:55,00:06:43.200000
4,10,J. LATVALA,1,-1.9,-1 days +23:59:58.100000,00:03:26.100000


In [40]:
dd,ddd = getSplitTimes(splittimes['SS2'])
dd.head()

,start,carNo,driverName,team,eligibility,stageTime,driverName_nationality,timestageTime
0,1.0,1,S. OGIER,M-SPORT WORLD RALLY TEAM,M,10:26.1,FRA,00:10:26.100000
1,2.0,5,T. NEUVILLE,HYUNDAI MOTORSPORT,M,10:22.3,BEL,00:10:22.300000
2,3.0,10,J. LATVALA,TOYOTA GAZOO RACING WRT,M,10:24.6,FIN,00:10:24.600000
3,4.0,2,O. TANAK,M-SPORT WORLD RALLY TEAM,M,10:21.9,EST,00:10:21.900000
4,5.0,6,D. SORDO,HYUNDAI MOTORSPORT,M,10:21.5,ESP,00:10:21.500000


In [41]:
ddd.head()

,carNo,driverName,Split,split_,tdsplit_,timesplit_
0,1,S. OGIER,1,00:03:28.0,00:00:00,00:03:28
1,1,S. OGIER,2,00:06:48.2,00:00:00,00:06:48.200000
2,5,T. NEUVILLE,1,-1.4,-1 days +23:59:58.600000,00:03:26.600000
3,5,T. NEUVILLE,2,-5.0,-1 days +23:59:55,00:06:43.200000
4,10,J. LATVALA,1,-1.9,-1 days +23:59:58.100000,00:03:26.100000


In [42]:
#split, tdsplit, timesplit

In [43]:
#StageTimes stagetimes
def _getStageTimesBase(path):
    ''' Get stage results and overall results at end of stage '''
    
    url='{base}{path}'.format(base=URL_BASE, path=path)
    results=loader(url)
    if not validateLoadedDF(results[0]): return DataFrame()

    cols=[0,1]     

    results[0].columns=['pos', 'carNo','driverName','time','diffPrev','diffFirst']
    results[1].columns=['pos', 'carNo','driverName','time','diffPrev','diffFirst']
    for i in cols:
        results[i].fillna(0,inplace=True)
        results[i]['pos']=results[i]['pos'].astype(float)
        for j in ['carNo']:
            results[i][j]=results[i][j].astype(str)

        results[i]['driverName'],results[i]['driverName_nationality']=_driverNameNationality(results[i])

    return results[0], results[1]#results[0].reset_index(drop=True), df_overallpart.reset_index(drop=True)

def getStageTimes( path):
    r = _getStageTimesBase(path)
    for df in r:
        if validateLoadedDF(df): 
            for col in ['time','diffPrev','diffFirst']:
                df['td_'+col]=df.apply(lambda x: regularTimeString(x[col]),axis=1)
    return r[0],r[1] 

In [44]:
stagetime,overalltime =getStageTimes( stagetimes['SS15'])

In [20]:
def _saveTable(conn, df, year, rally, table):
    if df.empty: return
    df['Year']=year
    df['Rally']=rally
    df.to_sql(table.lower(), conn, index=False, if_exists='append')

def saveItinerary(conn,lookup, year, rally, table='Itinerary'):
    df=getItinerary(lookup[table])
    _saveTable(conn,df, year, rally, table)
    
def savePenalties(conn,lookup, year, rally,table='Penalties'):
    df=getPenalties(lookup[table])
    _saveTable(conn,df, year, rally, table)
    
def saveRetirements(conn,lookup, year, rally, table='Retirements'):
    df=getRetirements(lookup[table])
    _saveTable(conn, df, year, rally, table)
    
def saveEntrylist(conn,lookup, year, rally, table='Startlists'):
    df=getEntryList(lookup[table])
    _saveTable(conn, df, year, rally, 'entrylist')
    
def saveStartlist(conn,lookup, year, rally, section, table='Startlists'):
    df=getEntryList(lookup[table])
    if df.empty: return
    df['section'] = section
    _saveTable(conn, df, year, rally, table)

def saveStageWinners(conn,lookup, year, rally, table='Stagewinners'):
    df=getStagewinners(lookup[table])
    _saveTable(conn, df, year, rally, table)

def saveOverall(conn, year, rally, overall, stage, table='overallstage'):
    df=getOverall(overall[stage])
    if df.empty: return
    df['stage'] = stage
    _saveTable(conn, df, year, rally, table)

def saveOverallFinal(conn, year, rally, overall, stage, table='overallfinal'):
    df=getOverallFinal(overall[stage])
    _saveTable(conn, df, year, rally, table)

def saveSplitTimes(conn,lookup, year, rally, splittimes, stage, table=('splittimesbase','splittimessplits')):
    df_splitTimes_base,df_splitTimes_splits = getSplitTimes(splittimes[stage])
    if not df_splitTimes_base.empty:
        df_splitTimes_base['stage'] = stage
    if not df_splitTimes_splits.empty:
        df_splitTimes_splits['stage'] = stage
    _saveTable(conn, df_splitTimes_base, year, rally, table[0])
    _saveTable(conn, df_splitTimes_splits, year, rally, table[1])
    
def saveStageTimes(conn,lookup, year, rally, stagetimes, stage, table=('stagetimes', 'overalltimes')):
    stagetime,overalltime = getStageTimes( stagetimes[stage])
    if not stagetime.empty: 
        stagetime['stage'] = stage
    if not overalltime.empty:
        overalltime['stage'] = stage
    _saveTable(conn, stagetime, year, rally, table[0])
    _saveTable(conn, overalltime, year, rally, table[1])

def saveChampionshipStandings(conn, year, rally, championship, typ='Drivers', table=None):
    df_standings = getChampionshipStandings(championship[typ]['path'], typ )
    
    if table is None: table='standings_{}'.format(typ.lower().replace('-',''))
    _saveTable(conn, df_standings, year, rally, table)
    
def fullScrape(year='2017', url = 'http://www.wrc.com/en/wrc/results/rally-results/page/3049----.html',
               dbname='fulldb.db'):
    conn = sqlite3.connect(dbname)
    rallies = getRalliesList(url)
    
    for rally in rallies:
        print('Grabbing data for {}'.format(rally))
        lookup = getRallyDetails(rallies, rally)
        lookupstarts = getStartinglistURLs(rallies,rally, lookup['Startlists'] )
        overall = getOverallURLs(rallies,rally, lookup['Overall'] )
        splittimes = getOverallURLs(rallies,rally, lookup['SplitTimes'] )
        stagetimes = getOverallURLs(rallies,rally, lookup['StageTimes'] )
        stagewinners = getOverallURLs(rallies,rally, lookup['Stagewinners'] )
        championship = getChampionshipURLs()
             
        saveChampionshipStandings(conn, year, rally, championship, 'Drivers')
        saveChampionshipStandings(conn, year, rally, championship, 'Co-Drivers')
        saveChampionshipStandings(conn, year, rally, championship, 'Teams')
        
        saveItinerary(conn,lookup, year, rally)
        savePenalties(conn,lookup, year, rally)
        saveRetirements(conn,lookup, year, rally)
        saveEntrylist(conn,lookup, year, rally)
        for section in lookupstarts:
            if section != 'Entry List':
                saveStartlist(conn,lookup, year, rally, section)
        saveStageWinners(conn,lookup, year, rally)
        for stage in overall:
            if stage=='Final Results':
                saveOverallFinal(conn, year, rally, overall, stage)
            else:
                saveOverall(conn, year, rally, overall, stage,)
        for stage in splittimes:
            print(stage)
            saveSplitTimes(conn,lookup, year, rally, splittimes, stage)
        for stage in stagetimes:
            try:
                saveStageTimes(conn,lookup, year, rally, stagetimes, stage)
            except:
                print('bailed on stagetimes {} {}'.format(rally, stage))

In [225]:
dbname='fulldb2.db'
!rm $dbname
fullScrape(dbname=dbname)


/en/wrc/results/monte-carlo/stage-times/page/318-226---.html Rallye Monte-Carlo
/en/wrc/results/sweden/stage-times/page/326-227---.html Rally Sweden
/en/wrc/results/mexico/stage-times/page/334-228---.html Rally Guanajuato México
/en/wrc/results/france/stage-times/page/400-236---.html Che Guevara Energy Drink Tour de Corse
/en/wrc/results/argentina/stage-times/page/346-230---.html YPF Rally Argentina
/en/wrc/results/portugal/stage-times/page/342-229---.html Vodafone Rally de Portugal
/en/wrc/results/italy/stage-times/page/360-231---.html Rally Italia Sardegna
/en/wrc/results/poland/stage-times/page/368-232---.html ORLEN 74th Rally Poland
/en/wrc/results/finland/stage-times/page/376-233---.html Neste Rally Finland
/en/wrc/results/germany/stage-times/page/384-234---.html ADAC Rallye Deutschland
/en/wrc/results/spain/stage-times/page/408-237---.html RallyRACC Catalunya - Rally de España
/en/wrc/results/wales/stage-times/page/416-238---.html Dayinsure Wales Rally GB 
/en/wrc/results/austral

/usr/local/lib/python3.6/site-packages/pandas/core/generic.py:1534: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)
/usr/local/lib/python3.6/site-packages/pandas/io/sql.py:473: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  chunksize=chunksize, dtype=dtype)


SS1
SS2
SS3
SS4
SS5
SS6
SS7
SS8
SS9
SS10
SS11
SS12
SS13
SS14
SS15
SS16
SS17
Grabbing data for sweden
SS1
SS2
SS3
SS4
SS5
SS6
SS7
SS8
SS9
SS10
SS11
SS12
SS13
SS14
SS15
SS16
SS17
SS18
bailed on stagetimes sweden SS12
Grabbing data for mexico
SS0
SS1
SS2
SS3
SS4
SS5
SS6
SS7
SS8
SS9
SS10
SS11
SS12
SS13
SS14
SS15
SS16
SS17
SS18
SS19
bailed on stagetimes mexico SS2
bailed on stagetimes mexico SS3
Grabbing data for france
SS1
SS2
SS3
SS4
SS5
SS6
SS7
SS8
SS9
SS10
Grabbing data for argentina
SS1
SS2
SS3
SS4
SS5
SS6
SS7
SS8
SS9
SS10
SS11
SS12
SS13
SS14
SS15
SS16
SS17
SS18
Grabbing data for portugal
SS1
SS2
SS3
SS4
SS5
SS6
SS7
SS8
SS9
SS10
SS11
SS12
SS13
SS14
SS15
SS16
SS17
SS18
SS19
Grabbing data for italy
SS1
SS2
SS3
SS4
SS5
SS6
SS7
SS8
SS9
SS10
SS11
SS12
SS13
SS14
SS15
SS16
SS17
SS18
SS19
Grabbing data for poland
SS1
SS2
SS3
SS4
SS5
SS6
SS7
SS8
SS9
SS10
SS11
SS12
SS13
SS14
SS15
SS16
SS17
SS18
SS19
SS20
SS21
SS22
SS23
bailed on stagetimes poland SS6
Grabbing data for finland
SS1
SS2
SS3
SS4
SS5

In [12]:
q="SELECT * FROM {}  LIMIT 3;".format('splittimesbase')
conn = sqlite3.connect('fulldb2.db')#('wrc_2017_full.db')
pd.read_sql_query(q,conn)

,start,carNo,driverName,team,eligibility,stageTime,driverName_nationality,timestageTime,stage,Year,Rally
0,1.0,1,S. OGIER,M-Sport World Rally Team,M,12:10.9,FRA,7.309000e+11,SS1,2017,monte-carlo
1,2.0,5,T. NEUVILLE,Hyundai Motorsport,M,12:10.2,BEL,7.302000e+11,SS1,2017,monte-carlo
2,NaN,4,H. PADDON,Hyundai Motorsport,M,,NZL,NaN,SS1,2017,monte-carlo


In [81]:
q="SELECT name FROM sqlite_master WHERE type='table';"

pd.read_sql_query(q,conn)

,name
0,itinerary
1,penalties
2,retirements
3,entrylist
4,startlists
5,stagewinners
6,overallstage
7,overallfinal
8,splittimesbase
9,splittimessplits


In [79]:
for i in ['standings_drivers','standings_co-drivers','standings_teams']:
    droptable(conn,i)

In [82]:
q="SELECT * FROM standings_drivers LIMIT 3;"

pd.read_sql_query(q,conn)

,pos,name,roundnum,name_nationality,Total,Round,bonusRound,posRound,pointsRound,rallyname,Year,Rally
0,1.0,Sébastien OGIER,1,,232.0,"25\n<span class=""newline"">\n ...",NaN,1.0,25.0,85e Rallye Monte-Carlo 2017,2017,monte-carlo
1,1.0,Sébastien OGIER,2,,232.0,"15+4\n<span class=""newline"">\n ...",4.0,3.0,15.0,Rally Sweden,2017,monte-carlo
2,1.0,Sébastien OGIER,3,,232.0,"18+4\n<span class=""newline"">\n ...",4.0,2.0,18.0,Rally Guanajuato México,2017,monte-carlo


In [11]:
!ls -al

total 31080
drwxr-xr-x@  8 ajh59  1182653967      272  8 Jan 16:36 .
drwxr-xr-x@ 62 ajh59  1182653967     2108  7 Jan 17:21 ..
drwxr-xr-x@  3 ajh59  1182653967      102  8 Jan 13:11 .ipynb_checkpoints
-rw-r--r--@  1 ajh59  1182653967  8331264  8 Jan 12:40 fulldb2.db
-rw-r--r--@  1 ajh59  1182653967   165771  8 Jan 16:36 scrapertest.ipynb
-rw-r--r--@  1 ajh59  1182653967   126976  8 Jan 02:35 test1.db
-rw-r--r--@  1 ajh59  1182653967  7266304  8 Jan 02:44 wrc_2017_full.db
-rw-r--r--@  1 ajh59  1182653967    20047  8 Jan 02:19 wrc_scraper.py


In [21]:
 df_standings = getChampionshipStandings(championship[typ]['path'], typ )

NameError: name 'getChampionshipStandings' is not defined

In [80]:
year = 2017
championship = getChampionshipURLs()
url = 'http://www.wrc.com/en/wrc/results/rally-results/page/3049----.html'
rallies = getRalliesList(url)
    
for rally in rallies:
    saveChampionshipStandings(conn, year, rally, championship, 'Drivers')
    saveChampionshipStandings(conn, year, rally, championship, 'Co-Drivers')
    saveChampionshipStandings(conn, year, rally, championship, 'Teams')


/en/wrc/results/monte-carlo/stage-times/page/318-226---.html Rallye Monte-Carlo
/en/wrc/results/sweden/stage-times/page/326-227---.html Rally Sweden
/en/wrc/results/mexico/stage-times/page/334-228---.html Rally Guanajuato México
/en/wrc/results/france/stage-times/page/400-236---.html Che Guevara Energy Drink Tour de Corse
/en/wrc/results/argentina/stage-times/page/346-230---.html YPF Rally Argentina
/en/wrc/results/portugal/stage-times/page/342-229---.html Vodafone Rally de Portugal
/en/wrc/results/italy/stage-times/page/360-231---.html Rally Italia Sardegna
/en/wrc/results/poland/stage-times/page/368-232---.html ORLEN 74th Rally Poland
/en/wrc/results/finland/stage-times/page/376-233---.html Neste Rally Finland
/en/wrc/results/germany/stage-times/page/384-234---.html ADAC Rallye Deutschland
/en/wrc/results/spain/stage-times/page/408-237---.html RallyRACC Catalunya - Rally de España
/en/wrc/results/wales/stage-times/page/416-238---.html Dayinsure Wales Rally GB 
/en/wrc/results/austral